In [2]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as pp
import datetime
%matplotlib inline
from sklearn.decomposition import PCA
from scipy.cluster.vq import kmeans, kmeans2
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist
import fastcluster

In [3]:
data.tail()

NameError: name 'data' is not defined

In [ ]:
data['Year'] = pd.DatetimeIndex(data['Discovery Date YYYY-MM-DD']).year
data['Month'] = pd.DatetimeIndex(data['Discovery Date YYYY-MM-DD']).month
data['Day'] = pd.DatetimeIndex(data['Discovery Date YYYY-MM-DD']).day
data.head()

# Cluster Analysis
###Version 1- no Datetime in cluster analysis

In [ ]:
df = data_stnd.iloc[:,2:8]
df.head()

In [ ]:
score = []
cluster_array = [kmeans(df, i) for i in range(1,10)]
pp.plot([var for (array,var) in cluster_array])
pp.show()

In [ ]:
kmeans2(df, 2)
pp.scatter(PCs[:, 0], PCs[:, 1], c = kmeans2(df,2)[1], cmap = 'bwr')

##Dendogram version 1

In [ ]:
fastcluster.linkage (df, method='single', metric='euclidean', preserve_input='True')

###Version 2- Datetime in cluster analysis

In [ ]:
df = data_stnd.iloc[:,2:8]
df['Year'], df['Month'], df['Day'] = data_stnd.iloc[:,10],data_stnd.iloc[:,11],data_stnd.iloc[:,12]
df.head()

In [ ]:
score = []
cluster_array = [kmeans(df, i) for i in range(1,10)]
pp.plot([var for (array,var) in cluster_array])
pp.show()

In [ ]:
kmeans2(df, 2)
pp.scatter(PCs[:, 0], PCs[:, 1], c = kmeans2(df,2)[1], cmap = 'bwr')

In [ ]:
fastcluster.linkage(df, method='single', metric='euclidean', preserve_input='True')